# Pyspark Guide - Basico
Por Alex Almeida Cordeiro
01/2020

## O básico

### Executando um Workbook

Execução efêmera, o conteúdo das variáveis será perdido
```python
dbutils.notebook.run('/comum/setup_conn_sadatalakevcbr.ipynb',60)
```

Execução semelhante a um include (uma célula somente com esse código):

```
%run "/comum/eng_shared_functions.ipynb"
```

### Exibindo o conteúdo de um dataframe:
```python
df.show(100) # lista 100 linhas
df.show(100, False) # lista 100 linhas, não trunca campos
```

### Mostrando estrutura de um dataframe
```python
df.printSchema()

```


### 
```python
```

## Recebendo parametros e retornando resultados
Recebe um parametro, converte para numerico, consiste no caso de parametro invalido, ao final do notebook termina passando Ok como retorno.

```python
dbutils.widgets.text("param_num", "")
try:
  my_param  = int(dbutils.widgets.get("param_num"))
  flag_no_param = False
except ValueError:
  my_param = 0
  flag_no_param = True
if my_param > 99:
  raise ValueError("Parametros invalidos")

muitas_operacoes_depois()

dbutils.notebook.exit("Ok")
  
```

## Funções utilitárias (dbutils.fs)

### Criar diretérios de forma recursiva
```python
dbutils.fs.mkdirs('my/path')
```


### Gravando um arquivo simples
Sim, ele vai gerar um arquivo simples, eu podia usar um dataframe ou RDD para isso? Podia mas ia gerar uma pasta com um monte de arquivos dentro.
```python
my_file_content = 'Hello!! Good by!!'
dbutils.fs.put('my/file/path/and/name.txt', my_file_content, overwrite = True) 
```

### Lendo um arquivo simples (texto)
Ok, está um pouco fora do lugar mas faz sentido estar aqui por causa da gravação
```python
rdd = spark.sparkContext.wholeTextFiles('my/file/path/and/name.txt')
my_file_content = rdd.collect()[0][1]
```

## Lendo e gravando
TODO: Terminar

### Lendo um dataset para um dataframe

#### json

```python
spark.read.json('my/folder')
```

#### csv
```python
spark.read.csv('my/folder')
spark.read.csv('my/folder', header=True) # com cabeçalho
```

#### parquet
```python
df = spark.read.format("parquet").load('my/folder')
```

#### delta
```python
df = spark.read.format("delta").load('my/folder')
```

### Utilizando um esquema predefinido
```python
spark.read.schema(my_schema).json(source_dir).createOrReplaceTempView('RawDetailed')
```

## criando um dataframe de teste

In [0]:
# https://dwgeek.com/replace-pyspark-dataframe-column-value-methods.html/
from pyspark.sql.functions import *

data = []
for ano in range(2020, 2023):
  for id in range (100):
    data.append ((ano, (ano - 2020) * 100 + id, str(id)))
df = spark.createDataFrame(data, ["year", "id", "d_id"])
df.show(5,False)

+----+---+----+
year|id |d_id|
+----+---+----+
2020|0 |0 |
2020|1 |1 |
2020|2 |2 |
2020|3 |3 |
2020|4 |4 |
+----+---+----+
only showing top 5 rows

# Operações com Dataframe

## Manipulando dataframes

### Preenche campos vazios, trata NAs:
Nesse exemplo troca os campos vazios de MyField1 por 0 e de MyField2 por 'NULO'
```python
my_df = my_df.fillna({'MyField1': 0, 'MyField2': 'NULO'})
```

### Eliminando colunas
TODO: testar se dá para fazer com uma lista

```python
my_df = my_df.drop(col('tag2')).drop(col('time2'))
```

### Criando um novo campo a partir da Substring de outro
Esse cria uma coluna ano com os primeiros quatro caracteres de uma data.
```python
df = df.withColumn('Ano', substring('Data', 1,4)) \
```
      

### Join
Efetuando o join entre dois datasets
```python
df_open_time = df_all.join(df_tag_list, (col('tag') == col('tag2')) & (col('time') >= col('time2')))
```

### Eliminando duplicidades por uma chave
Elimna duplicidades por tag e time.
```python
df_raw = df_raw.drop_duplicates(subset=['tag', 'time']) 
```

### Exemplos de agregação e seleção
Aqui filtramos uma coluna (time), agrupamos pela TAG agregando time pelo minimo, renomeamos tag para tag2 e a coluna resultante "min(time)" para time2
```python
df_tag_list = df_all.filter((col('Next_time') == maximum_timestamp)) \
              .filter((col('time') < '9999-12-31 99:99:99')) \
              .groupby('tag').agg({'time': 'min'}) \
              .select(col('tag').alias('tag2'), col('min(time)').alias('time2')) 
```

## Exemplos de manipulação avançada

### Clausula Window: diferença entre datas entre dois registros
Aqui vamos calcular a diferença em segundos de um campo data de um registro para o outro a partir de uma chave, ou seja, a partir do campo chave teremos a proxima data
Baseado em em https://www.arundhaj.com/blog/calculate-difference-with-previous-row-in-pyspark.html
```python
from pyspark.sql.window import Window
w_lag = Window.partitionBy("Chave").orderBy(desc('Time'))
my_df = my_df.withColumn("Next_time", lag(df_raw['Time']).over(w_lag))
my_df = my_df.withColumn( "Time_gap", df_raw['Next_time'].cast("long") - df_raw['Time'].cast("long"))
```


### Eliminando chave duplicada pelo menor timestamp
Aqui temos um exemplo onde filtramos um range de datas, criamos uma coluna de ranking por um timestamp agrupado por duas chaves (id_doc, id_item), depois filtramos onde ranking = 1 (somente o timestamp mais atual) e por fim eliminamos o ranking
```python
from pyspark.sql.window import Window

processing_start_date = '2021-01-01'
processing_end_date = '2021-01-05'
timestamp_field = 'ts_insert_raw'
key_fields = ['id_doc', 'id_item']
window = Window.partitionBy(key_fields).orderBy(col(timestamp_field).desc())
df = df.filter(col(raw_layer_date_filter_field)>= processing_start_date) \
      .filter(col(raw_layer_date_filter_field)<= processing_end_date) \
      .withColumn("rank_temp", rank().over(window)).filter(col("rank_temp") == 1).drop("rank_temp") 
```


### Monstrando contagem por mês/ano a partir de uma data
Ainda ordena decrescente
```python
df_DadosCPS.withColumn('ANO_MES_REMESSA', substring('DATA_REMESSA', 1,7)) \
  .groupBy('ANO_MES_REMESSA') \
  .agg({'RESULTADO':'avg', "*":'count'}) \
  .select('ANO_MES_REMESSA', col('avg(RESULTADO)').alias('MEDIA_RESULTADO'), col('count(1)').alias('QUANTIDADE')) \
  .sort(col('ANO_MES_REMESSA').desc()) \
  .show()
```

```
+---------------+------------------+----------+
|ANO_MES_REMESSA|   MEDIA_RESULTADO|QUANTIDADE|
+---------------+------------------+----------+
|        2020-12|29.810059438896857|      4206|
|        2020-11| 33.53508969653114|     19343|
|        2020-10|34.256294685788326|     23917|
|        2020-09| 36.06784174470488|     15911|
|        2020-08|              13.3|         1|
|        2020-07|              34.3|         4|
|        2020-05|             43.56|         1|
|        2020-04|              28.4|         1|
+---------------+------------------+----------+
```

## Exibição de dados

Montrando um dataframe formatado (data)

## Mais material

- http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.Column.alias

# SQL

## Criando tabelas e views

### Criando uma view a partir de um dataframe:

```python
my_df.createOrReplaceTempView('MyTable')
```

## Rodando código SQL
Depois que você cria tabelas e views pode rodar SQL normalmente usando %sql na celula, no pyspark pode rodar assim:

```python
my_df = spark.sql('SELECT * FROM MINHA_TABELA')
```

# Delta

## Operações com delta

### Verificando se uma pasta é delta
retorna True ou False
``` python
DeltaTable.isDeltaTable(spark, 'my/path')
```

## Delta: Finalizando o tratamento dos dados

### Otiminzando o tamanho dos arquivos
Elimina arquivos pequenos concatenando-os e gerando arquimos maiores
https://docs.databricks.com/delta/optimizations/file-mgmt.html#delta-optimize
```sql 
%sql
OPTIMIZE CleansedPiOsiDetails
```
Para rodar do pyspark (creio que não há uma função especifica):
```python
folder = '/path/to/data'
spark.sql("OPTIMIZE delta.`{0}`".format(folder))
```

### Elimina arquivos não usados 
Elimina arquivos não ativos, predemos o timetravel mas reduzimos consumo espaço, retem 7 dias. 
```sql 
%sql
VACUUM CleansedPiOsiDetails
```
```python
deltaTable.vacuum()     # vacuum files not required by versions more than 7 days old
deltaTable.vacuum(100)  # vacuum files not required by versions more than 100 hours old
```

## Time travel

Vamos supor que temos uma tabela delta
```python
delta_df = DeltaTable.forPath(spark, "/mnt/data/delta")
```

### Mostrando a lista de versões
```python
delta_df.history().show(10, True)
```


```
+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
|      2|2021-01-20 19:41:16|3998117388507181|alex.cordeiro.ac1...|    MERGE|[predicate -> (((...|null|[1481698408020379]|0114-184800-haste129|          1|WriteSerializable|        false|[numTargetRowsCop...|        null|
|      1|2021-01-20 19:03:00|3998117388507181|alex.cordeiro.ac1...|    MERGE|[predicate -> (((...|null|[1481698408020379]|0114-184800-haste129|          0|WriteSerializable|        false|[numTargetRowsCop...|        null|
|      0|2021-01-20 19:00:47|3998117388507181|alex.cordeiro.ac1...|    WRITE|[mode -> ErrorIfE...|null|[1481698408020379]|0114-184800-haste129|       null|WriteSerializable|         true|[numFiles -> 1, n...|        null|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
```

trazendo uma versão específica:
```python
version_1 = spark.read.format("delta").option("versionAsOf",1).load("/mnt/data/delta")
```

## Delta: Mais material
- https://docs.delta.io/latest/delta-utility.html#table-utility-commands

# Metadados

## Metadados - StructField
Documentação util:
- https://sparkbyexamples.com/pyspark/pyspark-structtype-and-structfield/

### Criando uma estrutura

```python
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([ 
    StructField("firstname",StringType(),True), 
    StructField("middlename",StringType(),True), 
    StructField("lastname",StringType(),True), 
    StructField("id", StringType(), True), 
    StructField("gender", StringType(), True), 
    StructField("salary", IntegerType(), True) 
  ])
```


### Verificando se um campo existe em um Dataframe
Se você deseja realizar algumas verificações nos metadados do DataFrame, por exemplo, se uma coluna ou campo existe em um DataFrame ou tipo de dados da coluna; podemos fazer isso facilmente usando várias funções em SQL StructType e StructField.
```python
print(df.schema.fieldNames.contains("firstname"))
print(df.schema.contains(StructField("firstname",StringType,true)))
```

### Alterando o metadado de um Dataframe
Também pode ser usado para renomear
```python
def set_metadata(my_metadata):
  my_metadata["Teste"] = "Ok"
  return my_metadata

new_df = df
new_df = new_df.select([col(c).alias(c, metadata = set_metadata(new_df.schema[c].metadata)  ) for c in new_df.columns])

for sc_col in new_df.schema:
  print (sc_col.jsonValue())
```
Também pode ser usado para renomear as colunas com:
```python
new_df = new_df.select([col(c).alias(c + "_new", metadata = set_metadata(new_df.schema[c])  ) for c in 
```

In [0]:
# https://dwgeek.com/replace-pyspark-dataframe-column-value-methods.html/
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType([ 
    StructField("firstname",StringType(),True, metadata = {"desc":"Nome"}), 
    StructField("middlename",StringType(),True), 
    StructField("lastname",StringType(),True), 
    StructField("id", StringType(), True), 
    StructField("gender", StringType(), True), 
    StructField("salary", IntegerType(), True) 
  ])

data = [("João", "Milho", "de Freitas", 1, "M", 2000),
        ("Maria", "de Lourdes", "de Freitas", 1, "M", 3000),
        ("Michael","", "Rose","40288","M",4000),
        ("Robert","","Williams","42114","M",4000),
        ("Maria","Anne","Jones","39192","F",4000),
        ("Jen","Mary","Brown","","F",3000),
       ]

df = spark.createDataFrame(data, schema )
df.show()
df.printSchema()

+---------+----------+----------+-----+------+------+
firstname|middlename| lastname| id|gender|salary|
+---------+----------+----------+-----+------+------+
 João| Milho|de Freitas| 1| M| 2000|
 Maria|de Lourdes|de Freitas| 1| M| 3000|
 Michael| | Rose|40288| M| 4000|
 Robert| | Williams|42114| M| 4000|
 Maria| Anne| Jones|39192| F| 4000|
 Jen| Mary| Brown| | F| 3000|
+---------+----------+----------+-----+------+------+

root
-- firstname: string (nullable = true)
-- middlename: string (nullable = true)
-- lastname: string (nullable = true)
-- id: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)

In [0]:
def set_metadata(my_metadata):
  my_metadata["Teste"] = "Ok"
  return my_metadata

new_df = df
new_df = new_df.select([col(c).alias(c, metadata = set_metadata(new_df.schema[c].metadata)  ) for c in new_df.columns])

for sc_col in new_df.schema:
  print (sc_col.jsonValue())


{'name': 'firstname', 'type': 'string', 'nullable': True, 'metadata': {'Teste': 'Ok', 'desc': 'Nome'}}
{'name': 'middlename', 'type': 'string', 'nullable': True, 'metadata': {'Teste': 'Ok'}}
{'name': 'lastname', 'type': 'string', 'nullable': True, 'metadata': {'Teste': 'Ok'}}
{'name': 'id', 'type': 'string', 'nullable': True, 'metadata': {'Teste': 'Ok'}}
{'name': 'gender', 'type': 'string', 'nullable': True, 'metadata': {'Teste': 'Ok'}}
{'name': 'salary', 'type': 'integer', 'nullable': True, 'metadata': {'Teste': 'Ok'}}

In [0]:
for sc_col in new_df.schema:
  if "newname" in sc_col.metadata and sc_col.metadata["newname"] != "" and sc_col.metadata["newname"] != sc_col.name:
    new_df = new_df.withColumnRenamed(sc_col.metadata["newname"], col(sc_col.name))
  
for sc_col in new_df.schema:
  print (sc_col.jsonValue())                             
new_df.show(2)
new_df.printSchema()

{'name': 'year_new', 'type': 'long', 'nullable': True, 'metadata': {'comment': 'deu certo', 'newname': 'year_new'}}
{'name': 'id_new', 'type': 'long', 'nullable': True, 'metadata': {'comment': 'deu certo', 'newname': 'id_new'}}
{'name': 'd_id_new', 'type': 'string', 'nullable': True, 'metadata': {'comment': 'deu certo', 'newname': 'd_id_new'}}
+--------+------+--------+
year_new|id_new|d_id_new|
+--------+------+--------+
 2020| 0| 0|
 2020| 1| 1|
+--------+------+--------+
only showing top 2 rows

root
-- year_new: long (nullable = true)
-- id_new: long (nullable = true)
-- d_id_new: string (nullable = true)

## Spark API - Metadados de tabelas
TODO: Checar & testar

Ver https://www.learningjournal.guru/courses/spark/spark-foundation-training/spark-data-types-and-metadata/

Esse código é scala???
```python
spark.catalog.listDatabases
// same as SHOW DATABASES
//This API gives you a dataset for the list of all databases. You can display the list using the show method.
spark.catalog.listDatabases.show
//You can collect it back to the master node as a Scala Array.
val dbs = spark.catalog.listDatabases.collect
//Then you can loop through the array and apply a function on each element. Let's apply the println.
dbs.foreach(println)
```

## Mais operações com metadados

### Como manter o metadado durante um ajuste
Ao ajustar uma coluna com o withColumn, por exemplo, todos os seus Metadados são perdidos.

```python
df = df.withColumn('NF', rtrim('NF').alias("", metadata = df.schema['NF'].metadata)) \
```

# Outros

## Outras operações interessantes

### Criando uma função aplicavel a um dataframe
Ver 
https://stackoverflow.com/questions/46667810/how-to-update-pyspark-dataframe-metadata-on-spark-2-1

```python
def withMeta(self, alias, meta):
    sc = SparkContext._active_spark_context
    jmeta = sc._gateway.jvm.org.apache.spark.sql.types.Metadata
    return Column(getattr(self._jc, "as")(alias, jmeta.fromJson(json.dumps(meta))))

Column.withMeta = withMeta

# new metadata:
meta = {"ml_attr": {"name": "label_with_meta",
                    "type": "nominal",
                    "vals": [str(x) for x in range(6)]}}

df_with_meta = df.withColumn("label_with_meta", col("label").withMeta("", meta))
```

## Erros e soluções

### TypeError: 'StructField' object is not callable usando a função col
Exemplo: df.filter(col('CAMPO') >= 1234).show()
Olhe no seu cósigo se você tem alguma função que usa uma variável chamada col...

## Mais material

- spark-gotchas -- https://github.com/awesome-spark/spark-gotchas/blob/master/06_data_preparation.md#python
- https://data.solita.fi/pyspark-execution-logic-code-optimization/
- https://medium.com/analytics-vidhya/getting-hands-dirty-in-spark-delta-lake-1963921e4de6